# Learning Goals
In Assignment 2, we learnt how to construct networks of spiking neurons and propagate information through a network of fixed weights. In this assignment, you will learn how to train network weights for a given task using brain-inspired learning rules.

Let's import all the libraries required for this assignment. 

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Question 1: Training a Network

## 1a. 
What is the purpose of a learning algorithm? In other words, what does a learning algorithm dictate, and what is the objective of it?

## Answer 1a. 
The learning algorithm is a set of instructions that allows a computer program to imitate the way a human gets better at characterizing some types of information. The main objective of learning algorithm is to enable system to learn and improve from experience without being explicitly programmed. It tries to train the model such that we can predict future outcomes in a way. The algorithm considers all the variables it has been exposed to during its training and finds the best combination of these variables to solve a problem. This unique combination of variables is ‘learned’ by the machine through trial and error. The purpose of  learning Algorithms  is to discover patterns in your data and then make predictions based on often complex patterns to answer business questions, detect and analyse trends and help solve problems.

## 1b. 
Categorize and explain the various learning algorithms w.r.t. biological plausibility. Can you explain the tradeoffs involved with the different learning rules? *Hint: Think computational advantages and disadvantages of biological plausibility.*

## Answer 1b. 
There are three main types of learning algorithms namely supervised learning, unsupervised learning and reinforcement learning. In supervised learning, we already know the output which are labels. We will try to map the input to output and train the model so that we can predict the outcome. In unsupervised learning, labels are not known and model will learn by finding patterns, and making clusters.

Speaking about their biological plausibility, we  can tell that :
Unsupervised learning has passive changes that exploits statsitical coorelations and it is useful for development i.e. wiring for receptive fields.
Reinforcement learning has conditioned changes that maximizes the rewards. That is useful for learning new behaviour.

# Question 2: Hebbian Learning

## 2a.

In this exercise, you will implement the hebbian learning rule to solve AND Gate. First, we need to create a helper function to generate the training data. The function should return lists of tuples where each tuple comprises of numpy arrays of rate-coded inputs and the corresponding rate-coded output. 

Below is the function to generate the training data. Fill the components to return the training data. 

In [2]:
def genANDTrainData(snn_timestep):
    """ 
    Function to generate the training data for AND 
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output
        
        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """
    
    #Initialize an empty list for train data
    train_data = []
    
    #encode 0. Numpy random choice function might be useful here. 
    zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
    
    #encode 1. Numpy random choice function might be useful here. 
    one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
    
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    i1 = np.array([zero,zero])
    i2 = np.array([zero,one])
    i3 = np.array([one,zero])
    i4 = np.array([one,one])
    
    case_1 = np.array([i1,zero])
    case_2 = np.array([i2,zero])
    case_3 = np.array([i3,zero])
    case_4 = np.array([i4,one])
    train_data = np.stack((case_1, case_2, case_3, case_4))

    return train_data

## 2b. 
We will use the implementation of the network from assignment 2 to create an SNN comprising of one input layer and one output layer. Can you explain algorithmically, how you can use this simple architecture to learn AND gate. Your algorithm should comprise of encoding, forward propagation, network training, and decoding steps. 

## Answer 2b.

Algorithm:

1) Network Architecture: AND Gate implementation is a lineraly separable function. We don't have to implement hidden layer. We can implement it with input and output layer. We will have two neurons in input layer and 1 in the output layer. We use spike based encoding to represent 0 and 1 and we also train the data set by feeding the output for given input.

2) Training the network:

- First, we initialise the network with random weights.
- We iterate over all the samples in traing data set and compute the mean firing rate
- we update the weight according to formula
- We run the model for certain number of epochs until convergence

3) Decoding: We decode the spike output using threshold volatge value

4) Testing Phase: We iterate over the train  data set with only inputs and compute the network output and decode it to the predict class and compare it with its label.

The SNN has already been implemented for you. You do not need to do anything here. Just understand the implementation so that you can use it in the later parts. 

In [3]:
class LIFNeurons:
    """ 
        Define Leaky Integrate-and-Fire Neuron Layer 
        This class is complete. You do not need to do anything here.
    """

    def __init__(self, dimension, vdecay, vth):
        """
        Args:
            dimension (int): Number of LIF neurons in the layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons
        
        """
        self.dimension = dimension
        self.vdecay = vdecay
        self.vth = vth

        # Initialize LIF neuron states
        self.volt = np.zeros(self.dimension)
        self.spike = np.zeros(self.dimension)
    
    def __call__(self, psp_input):
        """
        Args:
            psp_input (ndarray): synaptic inputs 
        Return:
            self.spike: output spikes from the layer
                """
        self.volt = self.vdecay * self.volt * (1. - self.spike) + psp_input
        self.spike = (self.volt > self.vth).astype(float)
        return self.spike

class Connections:
    """ Define connections between spiking neuron layers """

    def __init__(self, weights, pre_dimension, post_dimension):
        """
        Args:
            weights (ndarray): connection weights
            pre_dimension (int): dimension for pre-synaptic neurons
            post_dimension (int): dimension for post-synaptic neurons
        """
        self.weights = weights
        self.pre_dimension = pre_dimension
        self.post_dimension = post_dimension
    
    def __call__(self, spike_input):
        """
        Args:
            spike_input (ndarray): spikes generated by the pre-synaptic neurons
        Return:
            psp: postsynaptic layer activations
        """
        psp = np.matmul(self.weights, spike_input)
        return psp
    
    
class SNN:
    """ Define a Spiking Neural Network with No Hidden Layer """

    def __init__(self, input_2_output_weight, 
                 input_dimension=2, output_dimension=2,
                 vdecay=0.5, vth=0.5, snn_timestep=20):
        """
        Args:
            input_2_hidden_weight (ndarray): weights for connection between input and hidden layer
            hidden_2_output_weight (ndarray): weights for connection between hidden and output layer
            input_dimension (int): input dimension
            hidden_dimension (int): hidden_dimension
            output_dimension (int): output_dimension
            vdecay (float): voltage decay of LIF neuron
            vth (float): voltage threshold of LIF neuron
            snn_timestep (int): number of timesteps for inference
        """
        self.snn_timestep = snn_timestep
        self.output_layer = LIFNeurons(output_dimension, vdecay, vth)
        self.input_2_output_connection = Connections(input_2_output_weight, input_dimension, output_dimension)
    
    def __call__(self, spike_encoding):
        """
        Args:
            spike_encoding (ndarray): spike encoding of input
        Return:
            spike outputs of the network
        """
        spike_output = np.zeros(self.output_layer.dimension)
        for tt in range(self.snn_timestep):
            input_2_output_psp = self.input_2_output_connection(spike_encoding[:, tt])
            output_spikes = self.output_layer(input_2_output_psp)
            spike_output += output_spikes
        return spike_output/self.snn_timestep      

## 2c. 
Next, you need to write a function for network training using hebbian learning rule. The function is defined below. You need to fill in the components so that the network weights are updated in the right manner. 

In [4]:
def hebbian(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        w = np.zeros((1,2))
        #iterate over all samples in train_data
        for data in train_data:
            
            #compute the firing rate for the input
            r1 = np.sum(data[0][0])/len(data[0][0])
            r2 = np.sum(data[0][1])/len(data[0][1])

            #compute the firing rate for the output
            ro = np.sum(data[1])/len(data[1])

            #compute the correlation using the firing rates calculated above
            cor1 = r1 * ro
            cor2 = r2 * ro

            #compute the weight increment
            w[0][0] = lr * cor1
            w[0][1] = lr * cor2
            
            #increment the weight
            network.input_2_output_connection.weights += w

## 2d. 
In this exercise, you will use your implementations above to train an SNN to learn AND gate. 

In [26]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 5

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10

#print the initial weights
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_1 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(network_1, train_data, lr=2e-2, epochs=20)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_1(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_1(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_1(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_1(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_1.input_2_output_connection.weights)

initial weights  [[0.07416457 0.02400486]]
Testing 
zero  [1 0 0 0 0]
one  [0 0 1 1 1]
case 1 - output  [0.2]
case 2 - output  [0.4]
case 3 - output  [0.4]
case 4 - output  [0.6]
Final Output [case 1, case 2, case 3, case 4]  [0, 0, 0, 1]
Final Network Weights  [[0.44216457 0.39200486]]


/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' whe

# Question 3: Limitations of Hebbian Learning rule

## 3a. 
Can you learn the AND gate using 2 neurons in the output layer instead of one? If yes, describe what changes you might need to make to your algorithm in 2b. If not, explain why not, and what consequences it might entail for the use of hebbian learning for complex real-world tasks. 

## Answer 3a. 
Instead of one neuron in the output layer, we can also learn the AND gate with two neurons. We also have to use negative weights if two neurons are used. As a result, we can encode 0 as -0.5 and 1 as 0.5. So, negative weights are also considered. However, in a real-world scenario, this would be very computationally heavy.

## 3b. 
Train the network using hebbian learning for AND gate with the same arguments as defined in 2d. but now multiply the number of epochs by 20. Can your network still learn AND gate correctly? Inspect the initial and final network weights, and compare them against the network weights in 2d. Based on this, explain your observations for the network behavior. 

In [34]:
#Implementation for 3b. (same as 2d. but with change of one argument)
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10

print("initial weights ", input_2_output_weight)
network_2 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

hebbian(network_2, train_data, lr=2e-2, epochs=400)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_2(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_2(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_2(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_2(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_2.input_2_output_connection.weights)

initial weights  [[0.01100252 0.03182035]]
Testing 
zero  [0 1 1 0 1]
one  [1 1 1 1 0]
case 1 - output  [0.6]
case 2 - output  [1.]
case 3 - output  [1.]
case 4 - output  [0.8]
Final Output [case 1, case 2, case 3, case 4]  [1, 1, 1, 1]
Final Network Weights  [[7.37100252 7.39182035]]


## Answer 3b. 
The network has over saturated the weights which has lead to producing wrong results most of the times. This is because hebbian learning does not accomodate negative learning. As our epochs are 400, weights are increased as compared to earlier implemetation.

## 3c. 
Based on your observations and response in 3b., can you explain another limitation of hebbian learning rule w.r.t. weight growth? Can you also suggest a possible remedy for it?

## Answer 3c. 
In the hebbian learning rule, the update scheme for weights may result in very large weights when the number of iterations is large. This is another limitation of hebbian learning rule.
The remedy for this is Oja's rule. It is an extension of the hebbian learning rule. It is based on normalized weights, and the weights are normally normalized to unit length. This simple change results in a different but more general and stable weight update scheme compared to the classical Hebbian learning scheme.

## 3d. 
To resolve the issues with hebbian learning, one possibility is Oja's rule. In this exercise, you will implement and train an SNN using Oja's learning rule. 

In [35]:
def oja(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        w = np.zeros((1,2))
        #iterate over all samples in train_data
        for data in train_data:
            
            #compute the firing rate for the input
            r1 = np.sum(data[0][0])/len(data[0][0])
            r2 = np.sum(data[0][1])/len(data[0][1])

            #compute the firing rate for the output
            ro = np.sum(data[1])/len(data[1])

            #compute the correlation using the firing rates calculated above
            cor1 = r1 * ro
            cor2 = r2 * ro
            
            oja_term1 = network.input_2_output_connection.weights[0][0] * ro * ro
            oja_term2 = network.input_2_output_connection.weights[0][1] * ro * ro

            #compute the weight increment
            w[0][0] = lr * (cor1 - oja_term1)
            w[0][1] = lr * (cor2 - oja_term2)
            
            #increment the weight
            network.input_2_output_connection.weights += w

Now, test your implementation below. 

In [96]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 5

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10

#print the initial weights
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_3 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 3d. with the appropriate arguments
oja(network_3, train_data, lr=2e-2, epochs=10)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_3(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_3(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_3(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_3(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_3.input_2_output_connection.weights)

initial weights  [[0.05258029 0.01967894]]
Testing 
zero  [1 0 0 0 0]
one  [1 1 1 1 1]
case 1 - output  [0.2]
case 2 - output  [0.2]
case 3 - output  [0.4]
case 4 - output  [1.]
Final Output [case 1, case 2, case 3, case 4]  [0, 0, 0, 1]
Final Network Weights  [[0.27206337 0.24569904]]


/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' whe

# Question 4: Spike-time dependent plasticity (STDP)

## 4a. 
What is the limitation with hebbian learning that STDP aims to resolve?

## Answer 4a. 
In Hebbian learning, an association is strengthened when two events happen at around the same time (temporal order is not essential). STDP tries to solve this by decreasing weights. In Spike-timing-dependent plasticity(STDP), an association is strengthened when two events happen in the temporal order. Otherwise, it’s weakened. This is a timing dependent specialization of Hebbian learning.

## 4b. 
Describe the algorithm to train a network using STDP learning rule. You do not need to describe encoding here. Your algorithm should be such that its naturally translatable to a program. 

## Answer 4b. 
In STDP learning rule, there is a weight change if there is a pre-synaptic spike in the temporal vicinity of a post-synaptic spike. If the pre-synaptic spike occurs immediately before the post-synaptic spike, then the change is positive. Otherwise, the change is negative. Basically, our interest lies in the temporal range only which induces a change in the synaptic weights. In our network, each connection between the two neurons is associated with a delay of time units, which is the time difference between the post-synaptic firing time and the time the pre-synaptic potential starts rising. Since the model is envisioned to be used in digital systems, time is counted in discrete units. The membrane potential is described as a function of time and is increased by a synaptic weight value for each incoming spike. A constant value is subtracted from the membrane potential at every time instant to take into account the delay of time units. When the membrane potential crosses the threshold potential, the neuron produces a spike and the membrane potential decreases to the resting potential, which is the minimum potential level of a neuron.

## 4c. 
In this exercise, you will implement the STDP learning algorithm to train a network. STDP has many different flavors. For this exercise, we will use the learning rule defined in: https://dl.acm.org/doi/pdf/10.1609/aaai.v33i01.330110021. Pay special attention to Equations 2 and 3. 

Below is the class definition for STDP learning algorithm. Your task is to fill in the components so that the weights are updated in the right manner. 

In [97]:
class STDP():
    """Train a network using STDP learning rule"""
    def __init__(self, network, A_plus, A_minus, tau_plus, tau_minus, lr, snn_timesteps=20, epochs=30, w_min=0, w_max=1):
        """
        Args:
            network (SNN): network which needs to be trained
            A_plus (float): STDP hyperparameter
            A_minus (float): STDP hyperparameter
            tau_plus (float): STDP hyperparameter
            tau_minus (float): STDP hyperparameter
            lr (float): learning rate
            snn_timesteps (int): SNN simulation timesteps
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.  
            w_min (float): lower bound for the weights
            w_max (float): upper bound for the weights
        """
        self.network = network
        self.A_plus = A_plus
        self.A_minus = A_minus
        self.tau_plus = tau_plus
        self.tau_minus = tau_minus
        self.snn_timesteps = snn_timesteps
        self.lr = lr
        self.time = np.arange(0, self.snn_timesteps, 1)
        self.sliding_window = np.arange(-4, 4, 1) #defines a sliding window for STDP operation. 
        self.epochs = epochs
        self.w_min = w_min
        self.w_max = w_max
    
    def update_weights(self, t, i):
        """
        Function to update the network weights using STDP learning rule
        
        Args:
            t (int): time difference between postsynaptic spike and a presynaptic spike in a sliding window
            i(int): index of the presynaptic neuron
        
        Fill the details of STDP implementation
        """
        #compute delta_w for positive time difference
        if t>0:
            delta_w = self.A_plus * np.exp(-t / self.tau_plus)

        #compute delta_w for negative time difference
        else:
            delta_w = -self.A_minus * np.exp(-t / self.tau_minus)

        #update the network weights if weight increment is negative
        if delta_w < 0:
            change = self.lr * delta_w * (self.network.input_2_output_connection.weights - self.w_min)
            self.network.input_2_output_connection.weights += change 

        #update the network weights if weight increment is positive
        elif delta_w > 0:
            change = self.lr * delta_w * (self.w_max - self.network.input_2_output_connection.weights)
            self.network.input_2_output_connection.weights += change 
            
    def train_step(self, train_data_sample):
        """
        Function to train the network for one training sample using the update function defined above. 
        
        Args:
            train_data_sample (list): a sample from the training data
            
        This function is complete. You do not need to do anything here. 
        """
        input = train_data_sample[0]
        output = train_data_sample[1]
        for t in self.time:
            if output[t] == 1:
                for i in range(2):
                    for t1 in self.sliding_window:
                        if (0<= t + t1 < self.snn_timesteps) and (t1!=0) and (input[i][t+t1] == 1):
                            self.update_weights(t1, i)
    
    def train(self, training_data):
        """
        Function to train the network
        
        Args:
            training_data (list): training data
        
        This function is complete. You do not need to do anything here. 
        """
        for ee in range(self.epochs):
            for train_data_sample in training_data:
                self.train_step(train_data_sample)

Let's test the implementation

In [100]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 5

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_4 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
STDP_obj = STDP(network_4, A_plus = 0.6, A_minus = 0.3, tau_plus = 8, tau_minus = 5, lr = 0.25, 
                snn_timesteps=5, epochs=30, w_min=0, w_max=1)

#Train the network using STDP
STDP_obj.train(train_data)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_4(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_4(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_4(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_4(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_4.input_2_output_connection.weights)

initial weights  [[0.03690503 0.08065125]]
Testing 
zero  [0 0 0 0 1]
one  [1 1 1 1 1]
case 1 - output  [0.2]
case 2 - output  [0.4]
case 3 - output  [0.4]
case 4 - output  [1.]
Final Output [case 1, case 2, case 3, case 4]  [0, 0, 0, 1]
Final Network Weights  [[0.30058436 0.30058436]]


/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' whe

# Question 5: OR Gate
Can you train the network with the same architecture in Q2-4 for learning the OR gate. You will need to create another function called genORTrainData. Then create an SNN and train it using STDP. 

In [101]:
#Write your implementation of genORTrainData here. 

def genORTrainData(snn_timestep):
    
    #Initialize an empty list for train data
    training_data = []
    
    #encode 0. Numpy random choice function might be useful here. 
    zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
    
    #encode 1. Numpy random choice function might be useful here. 
    one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
    
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    i1 = np.array([zero,zero])
    i2 = np.array([zero,one])
    i3 = np.array([one,zero])
    i4 = np.array([one,one])
    
    case_1 = np.array([i1,zero])
    case_2 = np.array([i2,one])
    case_3 = np.array([i3,one])
    case_4 = np.array([i4,one])
    training_data = np.stack((case_1, case_2, case_3, case_4))

    return training_data

In [125]:
#Train the network for OR gate here using the implementation from 4c. 

#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.5

#Define a variable for snn timesteps
snn_timestep = 5

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_5 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
training_data = genORTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
OR_obj = STDP(network_5, A_plus = 0.6, A_minus = 0.3, tau_plus = 8, tau_minus = 5, lr = 0.25, 
                snn_timesteps=5, epochs=30, w_min=0, w_max=1)

#Train the network using STDP
OR_obj.train(training_data)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_5(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_5(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_5(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_5(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_5.input_2_output_connection.weights)

initial weights  [[0.079905   0.06529905]]
Testing 
zero  [0 0 1 0 0]
one  [1 1 1 1 1]
case 1 - output  [0.2]
case 2 - output  [0.6]
case 3 - output  [0.6]
case 4 - output  [1.]
Final Output [case 1, case 2, case 3, case 4]  [0, 1, 1, 1]
Final Network Weights  [[0.37777207 0.37777207]]


/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/koko/system/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' whe